In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
import numpy as np
    
def token_lookup():
    token_dict = {".":"<period>", ",": "<comma>", ";":"<semicolon>", ":":"<colon>", "\"":"<qoutation>", "'":"qoutation", "!":"<exclamation>", "?":"<question>", 
                  "(": "<leftparan>", ")": "<rightparan>", "{":"leftbrace", "}":"<rightbrace>", "[":"<leftbracket>", "]":"<rightbracket>", "--":"<dash>", "-":"<hyphen>",
                  "\n":"<return>"}
    return token_dict

token_dict = token_lookup()

english_text = open("./plot.tok.gt9.5000").read()#open("./plot_quote.10000").read()

for key, token in token_dict.items():
    english_text = english_text.replace(key, ' {} '.format(token))
english_text = english_text.lower()
english_text = english_text.split()
english_text = ' '.join(english_text)

english_text = english_text.split("<period>")
english_text = pd.DataFrame(english_text, columns=["sentences"])

#hindi_text = pd.read_csv("./hindi_train.csv").drop(columns=['experience']).iloc[:50]
mix_text = open("./codemix.txt").read()

for key, token in token_dict.items():
    mix_text = mix_text.replace(key, ' {} '.format(token))
mix_text = mix_text.lower()
mix_text = mix_text.split()
mix_text = ' '.join(mix_text)

mix_text = mix_text.split("<return>")
mix_text = pd.DataFrame(mix_text, columns=["sentences"])
mix_text.head

<bound method NDFrame.head of                                                sentences
0      the biggest drawback of इस योजना is that इस सब...
1                 दिल्ली is an urban territory <period> 
2       there banks are in बैंक बहुतायत and most of अ...
3       यही बात can also be said about आधार कार्ड भी ...
4       but various complaints are आ रही हैं up in ot...
...                                                  ...
25583   seven years i यह thinking hockey वपिस came कि...
25584                   लेकिन today फिर i lost <period> 
25585                 you मुझसे तकलीफ़ नहीं है <period> 
25586                                 you तकलीफ़ इस बात 
25587                                                   

[25588 rows x 1 columns]>

In [ ]:
en_mix = mix_text #pd.concat([english_text, mix_text], axis=0)
#en_mix = en_mix.sample(frac = 1)
en_mix.head()

,sentences
0,the biggest drawback of इस योजना is that इस सब...
1,दिल्ली is an urban territory <period>
2,there banks are in बैंक बहुतायत and most of अ...
3,यही बात can also be said about आधार कार्ड भी ...
4,but various complaints are आ रही हैं up in ot...


In [ ]:
def create_lookup_tables(text_df):
    vocab = set()
    for line in text_df['sentences']:
        vocab.update(set(line.split()))
    #vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return (vocab_to_int, int_to_vocab)

vocab_to_int, int_to_vocab = create_lookup_tables(en_mix)

train_int_text = []

for line in en_mix['sentences']:
    train_int_text.append([vocab_to_int[word] for word in line.split()])

train_int_text = [item for sublist in train_int_text for item in sublist]
print(train_int_text[:1000])

[28175, 21394, 5607, 10267, 211, 372, 24666, 7957, 211, 14437, 24666, 69, 1484, 27572, 7268, 13040, 23090, 520, 13229, 3194, 23452, 24666, 2138, 9041, 20069, 3194, 19376, 22974, 11723, 6627, 5065, 4417, 2156, 6946, 10267, 9245, 19299, 19692, 1012, 4485, 7223, 17379, 3194, 15747, 8120, 27733, 1012, 1471, 6517, 5152, 894, 17515, 1251, 3194, 12678, 25775, 8839, 11723, 5196, 6141, 149, 13040, 6627, 4398, 2099, 10267, 12867, 3194, 28175, 6946, 22405, 2975, 9572, 8120, 24666, 7957, 28175, 848, 10460, 9961, 28363, 10257, 1469, 21644, 4351, 25995, 5706, 28175, 22309, 10267, 151, 3194, 20570, 10257, 12128, 17144, 19451, 4923, 28175, 20658, 9732, 3194, 28175, 9732, 10267, 27345, 11364, 10267, 11283, 20756, 24666, 24894, 17144, 3194, 6627, 28175, 10123, 18179, 10267, 13255, 1251, 1012, 13026, 23681, 2156, 24236, 2592, 20350, 14069, 26305, 15411, 18138, 3911, 3194, 6627, 15250, 10257, 28175, 27936, 10267, 24540, 23215, 9822, 1625, 14939, 28190, 16282, 894, 10267, 894, 17515, 3194, 19376, 11723, 12

In [ ]:
print(en_mix.shape, len(vocab_to_int), max(train_int_text))

(25588, 1) 28383 28382


In [ ]:
#import helper

#data_dir = './brown.txt'

#helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

from distutils.version import LooseVersion
import warnings
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import tensorflow as tf2
#import helper
import numpy as np

#int_text, val_int_text, test_int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

def get_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learn_rate = tf.placeholder(tf.float32, name='learn_rate')
    return (inputs, targets, learn_rate)
    
def get_init_cell(batch_size, rnn_size):
    def build_cell(lstm_size):
        # Use a basic LSTM cell
        lstm = tf2.compat.v1.nn.rnn_cell.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf2.compat.v1.nn.rnn_cell.DropoutWrapper(lstm)#, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    num_layers = 2
    cell = tf2.compat.v1.nn.rnn_cell.MultiRNNCell([build_cell(rnn_size) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, dtype=tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")
    return (cell, initial_state)
    
def get_embed(input_data, vocab_size, embed_dim):
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed
    
def build_rnn(cell, inputs):
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)#,initial_state=initial_state)
    final_state = tf.identity(final_state, name="final_state")
    return outputs, final_state
    
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    
    logits = tf2.compat.v1.layers.dense(outputs, vocab_size, activation=None)
    return logits, final_state
    
def get_batches(int_text, batch_size, seq_length):
    words_per_batch = batch_size * seq_length
    num_batch = len(int_text)//words_per_batch
    
    input_text = np.array(int_text[:(num_batch*words_per_batch)])
    target_text = np.array(int_text[1:(num_batch * words_per_batch) + 1])

    target_text[-1] = input_text[0]
    
    x_batches = np.split(input_text.reshape(batch_size, -1), num_batch, 1)
    y_batches = np.split(target_text.reshape(batch_size, -1), num_batch, 1)
    
    batches = np.array(list(zip(x_batches, y_batches)))
    return batches
    
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 100
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 50

save_dir = '/content/drive/MyDrive/nlp_project/models/model_1_T12/save'

batches = get_batches(train_int_text, batch_size, seq_length)

In [ ]:
!pip install tensorflow_addons
import tensorflow_addons as tfa


train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = tfa.seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    
#batches = get_batches(train_int_text, batch_size, seq_length)
#train_graph = tf.Graph()
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')
    
# Save parameters for checkpoint
#helper.save_params((seq_length, save_dir))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:702: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/pyth

Epoch   0 Batch    0/307   train_loss = 10.254
Epoch   0 Batch   50/307   train_loss = 8.154
Epoch   0 Batch  100/307   train_loss = 8.065
Epoch   0 Batch  150/307   train_loss = 7.916
Epoch   0 Batch  200/307   train_loss = 7.867
Epoch   0 Batch  250/307   train_loss = 7.524
Epoch   0 Batch  300/307   train_loss = 7.554
Epoch   1 Batch   43/307   train_loss = 7.219
Epoch   1 Batch   93/307   train_loss = 7.199
Epoch   1 Batch  143/307   train_loss = 7.089
Epoch   1 Batch  193/307   train_loss = 7.091
Epoch   1 Batch  243/307   train_loss = 7.052
Epoch   1 Batch  293/307   train_loss = 6.820
Epoch   2 Batch   36/307   train_loss = 6.730
Epoch   2 Batch   86/307   train_loss = 6.533
Epoch   2 Batch  136/307   train_loss = 6.422
Epoch   2 Batch  186/307   train_loss = 6.495
Epoch   2 Batch  236/307   train_loss = 6.509
Epoch   2 Batch  286/307   train_loss = 6.441
Epoch   3 Batch   29/307   train_loss = 6.324
Epoch   3 Batch   79/307   train_loss = 6.125
Epoch   3 Batch  129/307   train_

In [ ]:
import tensorflow as tf
import numpy as np
#import helper

def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    index = np.where(probabilities == np.max(probabilities))[-1][0]

    return int_to_vocab[index]

#_,_,_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = 15, '/content/drive/MyDrive/nlp_project/models/model_1_T12/save'
def get_tensors(loaded_graph):
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    prob_tensor = loaded_graph.get_tensor_by_name("probs:0")

    return input_tensor, initial_state_tensor, final_state_tensor, prob_tensor
    
#test_text = [int_to_vocab[word] for word in test_int_text]
#test_text = " ".join(test_text)
#test_sent_text = test_text.split("<period>")

'''
test_sentences = input("Enter sentence:")
token_dict = token_lookup()
for key, token in token_dict.items():
	test_sentences = test_sentences.replace(key, ' {} '.format(token))
test_sentences = test_sentences.lower()
test_sentences = test_sentences.split("<period>")
'''
#all_perp = []

loaded_graph = tf.Graph()
with tf.compat.v1.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.compat.v1.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)
    #print(final_state)
    # Sentences generation setup
    gen_length = 4000
    prime_word = 'यही' #'moe_szyslak'
    gen_sentences = [prime_word]
            
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])
          
        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})

        #pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)

    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        if key=='.':
            tv_script = tv_script.replace(' ' + token.lower(), '\n')
        else:
            ending = ' ' if key in ['\n', '(', '"'] else ''
            tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/nlp_project/models/model_1_T12/save
यही नही ज्वार could be grown through the near market or कच्चे क्षेत्रों
for एक मनुष्य पौधों mostly हल्दी can be प्रयोग through lpg and बीज प्रयोग could be done
during the rows of season is very small and the first sown that पकने तो could हो सकती है taste
मुख्य कार्य of इस देश is 7 thousand land
in इस क्षेत्र लम्बाई of प्याज is carried on एक अवस्था
by making alert in normal areas still more than 2 c
m
long, planning as a variety became great
director china seems स्थापना भारतीय प्रशिक्षण were आयोजित on the central characters of 1980- munna, 2010 became'' आवारा दिवसीय हरिश्चंद्र', which became मुख्य प्रशिक्षण
इस प्रशिक्षण were महत्वपूर्ण योगदान
which आयोग has सिफारिश that there are काम for कृषि योग्य भूमि
for इस योजना the impact of कृषि भूमि and agricultural research is बढ़ जाता है up
by the research institute by the national farmer union, weekly and rural affairs have been successful in o